### Amazon S3 task

In [2]:
# 1) create sparse file with target size
# 2) upload file to S3 by partitioning file into chunks and use multi-file upload with return key, part_no etc..
# 3) copy file from one bucket to another by reassigning keys to point to new bucket

In [3]:
import moto
from moto import mock_s3
import boto3
from boto3.s3.transfer import TransferConfig


def create_sparse_file(filename: str, target_size: float):
    with open(filename, "wb") as out:
        out.truncate(target_size * 1024 ** 3)

        
@mock_s3
class AmazonAws:
    # set upload threshold to 100Mb
    _PART_THRESHOLD: int = 100 * (1024**2)
    connection = None
        
    def __init__(self):
        self.config = TransferConfig(multipart_threshold=self._PART_THRESHOLD)

        self._connect()
        
    def upload_file(self, path: str, bucket: str, key: str):
        print(f"Bucket_list before upload {self.get_buckets}")
        if bucket not in self.get_buckets:
            self.create_bucket(bucket)
        
        bucket_a_files_before = list(file for file in self.connection.Bucket(bucket).objects.all())
        print(f"Content of bucket_a before upload{bucket_a_files_before}\n")

        self.connection.meta.client.upload_file(path,
                                                bucket,
                                                key,
                                                Config=self.config)
        bucket_a_files_after = list(file for file in self.connection.Bucket(bucket).objects.all())
        print(f"Content of bucket_a after upload{bucket_a_files_after}\n")
        
    
    def move_content(self, source: str, target: str, key: str):
        # reassign pointers to new bucket instead of physically copying the file        
        copy_source = {'Bucket': source,
                       'Key': key}
        self.connection.meta.client.copy_object(CopySource=copy_source,
                                                Bucket=target,
                                                Key=key)
        
        print(f"Bucket list after content copy {self.get_buckets}\n")
        
        bucket_b_files = list(file for file in self.connection.Bucket(target).objects.all())
        print(f"Content of bucket_b {bucket_b_files}\n")
  
    @property
    def get_buckets(self):
        return list(self.connection.buckets.all())
    
    def create_bucket(self, name: str, **kwargs):
        self.connection.create_bucket(Bucket=name, **kwargs) 
    
    def _connect(self):
        self.connection = boto3.resource('s3',
                                         region_name='us-east-1')

In [ ]:
create_sparse_file(filename = 'test.file',
                   target_size = 5.0)

mock = mock_s3()
mock.start()

aws = AmazonAws()
aws.upload_file(path="test.file",
                bucket="bucket_a",
                key="test.file")
aws.create_bucket(name="bucket_b")
aws.move_content(source="bucket_a",
                 target="bucket_b",
                 key="test.file")
mock.stop()